Cluster Etiquette and Conda Training December 5th, 2018
========================================================

## I. Jupyter Notebook or What is this thing I'm looking at?

This is a [jupyter](http://jupyter.org/) notebook.

>The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and narrative text. Uses include: data cleaning and transformation, numerical simulation, statistical modeling, data visualization, machine learning, and much more.

We're going to use it because it makes combining explanation, examples, and exercises as easy as possible. Code cells directly run bash commands on Waffles and you (by aunthenticating with this site) will have access to all the files you would normally. 

### Tools
Click on the tabs on the left hand side of the notebook to open the following:
* Files: Displays the file system 
* Running: Shows what notebooks and terminals are running
* Commands: A compendium of useful commands for jupyter
* Cell Tools: For adapting jupyter notebooks to presentations
* Tabs: Navigation between open notebooks and terminals
  * Also above the active notebook
* Contents: Table of Contents with clickable links!

### Vocabulary to know
* **Cells**
 * Two kinds of containers for text
 * *Markdown*
    * Fancy formatted text
    * Running markdown cells formats the text according to the tags within
 * *Code*  
     * Fancy runnable code
     * Running code cells sends the commands within to the kernel 
     * Results are shown by default below the code cell
     * Number in brackets indicates the order the cell was executed on the kernel
* **Kernel**
 * The calculation engine that runs code
 * Make sure yours says *Bash* in the upper right
 * Code cells that are run are termed "inputs" to the kernel (In[*])
* **Modes**
    * *Command* - navigate between cells, modify them, run them
    * *Edit* - edit text within the code and markdown cells  

That's good enough for now. 

### Commands you'll use
* Up and Down arrows - navigate between cells
* Enter - Enter *Edit* mode in selected cell (double click a cell to do this as well) 
* Ctrl + Enter - Run Cell
* Shift + Enter - Run Cell and select next
* Esc - Escape and return to *Command* mode

## II. Housekeeping

Let's get started by making a nice folder for today's workshop on your personal drive. If you've done the pre-requisites for this course you may already have one. 

In [1]:
# Housekeeping Exercise 0.1
mkdir 20181205_conda-training
cd 20181205_conda-training

Open a new terminal to use later:
*File->New->Terminal*

Run the next code block to copy files off of temporary to your new folder (ctrl+enter with cell selected)

In [3]:
# Housekeeping Exercise 0.2
cp /Drives/W/Temporary/conda-workshop-2018/*.* ./
ls

cli-cheatsheet.pdf  conda-cheatsheet.pdf  jupyter-cheatsheet.pdf  python2.txt


## III. Cluster Etiquette
### Introduction
Remember the tragedy of the commons? The cluster, Waffles 2.0 is our local computing commons. 

>The tragedy of the commons is a term used in social science to describe a situation in a shared-resource system where individual users acting independently according to their own self-interest behave contrary to the common good of all users by depleting or spoiling that resource through their collective action.

Waffles isn't a beautiful pasture for us to despoil, but it is a limited resource that works best when everyone acts for their own interests while respecting the interests of the collective. 

What that means in practice:
1. Use the available job management tools from Slurm
2. Use the appropriate resources to those jobs


### 1. Job Management
 
No *Analysis* on the head node!
 
Must use the Slurm commands ```sbatch``` and ```srun``` as these parcel out jobs to the worker nodes. Watch them run with ```sq``` and review them with ```sacct```.

#### **sbatch** submits jobs to run in the background
* No user interaction possible 
* Many jobs can run at once
* Numerous options available to the user to customize

#### **srun** actively runs jobs
* Runs in active terminal
* One job per terminal
* Not as dependable as ```sbatch```

#### **sq** displays running job information
* Short for ```squeue```
* Shows **all** running jobs
* Find your job with ```sq | grep USERNAME```
* Watch your job with ```watch "sq | grep USERNAME"```
  * Quit the watch with CTRL+C

#### **sacct** displays finished job information
Note that it does not tell you what was submitted in --wrap="" so keep notes!  
* ```sacct -j JOBID``` for the basics of  
JobID    JobName  Partition    Account  AllocCPUS      State ExitCode

* ```sacct -e``` for the available columns
* ```sacct -j JOBID --format="NCPUS,MaxRSS,CPUTime"``` 
  * Use ```--format=""``` to show specific columns


We'll focus on ```sbatch``` for submission because the majority of optimization can be done with it.

#### Basic structure of ```sbatch```

```
sbatch -p PARTITION -c NCPUs --mem=MEMORY --wrap="ACTUAL COMMAND"
```

* ```--wrap```: Place the call to your tool in here
  * Don't forget to only use single quotes within the double quotes of ```wrap```  


* ```--mem```: How much memory does each job need?
 * Requests RAM resources 
 * Units must be specified along with quantity:
  * 1G = 1 Gigabyte
  * 1M = 1 Megabyte 


* ```-c```: The number of cores (aka. CPUs) requested for each job
  * Always use 1 core unless your tool specifically allows multi-threading / parallel processes
  * If your tool has multithread capability you have to match the number of cores called within ```wrap``` with the number of cpus specified by ```-c```  


* ```-p```: The partition to which this job will be submitted. One of: 
  * OutbreakResponse
  * Reference/Surveillance
  * NMLResearch
  * ExternalResearch

Always exports a slurm-JOBID.out file from the job that was completed. That JOBID number is important for any future troubleshooting.

In [1]:
# Exercise 1.1
# Run the following sbatch command
sbatch -p NMLResearch -c 1 --mem=1M --wrap="echo 'done'"


Submitted batch job 2787968


In [2]:
# Exercise 1.2
# Modify the below command with your job ID and run

more slurm-2787968.out

done


#### Timing

**Start Times**
Assembling 7500 genomes? Don't start it at 0800 because it will blow up Waffles (figuratively) and won't be done before you leave anyhow. Use the ```--begin``` flag to tell your job to start in the evening and be ready for morning!
>  --begin=16:00  
   --begin=now+1hour  
   --begin=2018-12-05T10:30:00  
   --begin=midnight  
   --begin=fika ( 3 pm)  
   --begin=teatime (4 pm)  


**Maximum Times**
Trying to align 100 E.coli genomes with ClustalW? That's not going to ever finish no matter how many resources you give it. Jobs on Waffles will now have a 24hr default max time. Need longer for your analysis?  
```--time``` is on your side.
> A time limit of zero requests that no time limit be imposed (don't abuse this)  
    Time limits can be given in the following formats:  
        --time=days-hours  
        --time=days-hours:minutes  
        --time=hours:minutes:seconds  
        --time=minutes:seconds  
        --time=0 - No limit  




In [4]:
## Exercise 2.1
# Run this cell with ctrl+enter then move on to 2.2 while the job completes
# Note that it has a time limit of 1 minute

sbatch -p NMLResearch -c 1 --mem=1G --time=01:00 --wrap="sleep 90; touch test90.out"



Submitted batch job 2787974


In [1]:
# Exercise 2.2
# Modify the maximum time from the last command so it actually finishes. 
# Don't forget time has to be given in the form of:
# "minutes", "minutes:seconds", "hours:minutes:seconds", 
# "days-hours", "days-hours:minutes", or "days-hours:minutes:seconds"
# Verify that test90.out has been written to your folder
# Don't worry about the -o argument for right now

sbatch -p NMLResearch -o sleepytest.out -c 1 --mem=1G --time=02:00 --wrap="echo 'Starting sleepytest'; sleep 45; echo 'Halfway there...';sleep 45; echo 'Done'; touch test90.out"


Submitted batch job 2992740


In [2]:
pwd

/Drives/K/AZetner/jupyter_notebooks


In [ ]:
# Optional Exercise 2.2b
# Open a bash terminal (File->New->Terminal)
# cd into your course folder and enter the below command
# tail -f sleepytest.out
# Quit with ctrl+c


In [6]:
# Exercise 2.3
# Modify the below sbatch command to start 1 minute from now

sbatch -p NMLResearch -c 1 --mem=1G --begin=now+1minute --wrap="touch waitstart.out"

Submitted batch job 2788004


In [7]:
## Exercise 2.4
# Add your job ids to the below sacct command and run this cell

# Job from 2.1.
sacct -j 2787974

# Job from 2.2
sacct -j 2787981

# Job from 2.3
sacct -j 2788004

# Note that these can be comma separated too:
# sacct -j JOBID1,JOBID2,JOBID3

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
2787974            wrap NMLResear+                     1    TIMEOUT      0:0 
2787974.bat+      batch                                1  CANCELLED     0:15 
       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
2787981            wrap NMLResear+                     1  COMPLETED      0:0 
2787981.bat+      batch                                1  COMPLETED      0:0 
       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
2788004            wrap NMLResear+                     1  COMPLETED      0:0 
2788004.bat+      batch                                1  COMPLETED      0:0 


#### Naming Jobs and Outputs

**Naming Jobs**

Give future you a break: name your jobs and don't have them all named 'wrap' as they are above

```--job-name``` or ```-J```

This helps to figure out which job is which when running several concurrently.


**Naming Outputs**

Redirects the standard output into a named file instead of the default of ```slurm-JOBID.out```

```--output``` or ```-o```

Fancy ways to change the output:
 * ```%j``` to add the job id
 * ```%u``` to add the user id
 * ```%x``` to add the job name
 
 eg. ```%u-%x-ID%j.out``` becomes ```azetner-SpadesAssembly-ID######.out```


In [9]:
# Exercise 3.1
# Edit all parts before running cell
# Rename this job with your username and fix the sq command so it shows the status of that job
sbatch -p NMLResearch -c 1 --mem=1G -J azetner-Is-Testing-Naming-Jobs --wrap="sleep 30"

sleep 5

sq | grep azetner
# or
squeue -u azetner




Submitted batch job 2788007
2788007  NMLResear  azetner-Is-Testing-Naming-Jobs                      azetner   RUNNING  0:05        UNLIMITED     1      waffles-b-3       1     1G


In [11]:
# Exercise 3.2
# Edit all parts before running cell
# Choose an appropriate partition on which to run this job
# Edit the job output name to include your username instead of mine

sbatch -p NMLResearch -c 1 --mem=1G -o azetners-file-list.out --wrap="ls"

sleep 10

ls -lr

# Check to see that your output file has been created


Submitted batch job 2788023
total 952
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:47 waitstart.out
drwx------. 3 azetner grp_azetner    21 Dec  1 18:35 testing
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:45 test90.out
drwx------. 2 azetner grp_azetner    45 Nov 29 11:58 sources
-rwx---r-x. 1 azetner grp_azetner 11406 Nov  2 16:31 snakemake_basecall_tutorial.ipynb
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:50 slurm-2788007.out
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:50 slurm-2788006.out
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:47 slurm-2788004.out
-rwx------. 1 azetner grp_azetner   106 Dec  1 18:44 slurm-2787974.out
-rwx------. 1 azetner grp_azetner   106 Dec  1 18:43 slurm-2787971.out
-rwx------. 1 azetner grp_azetner     5 Dec  1 18:41 slurm-2787968.out
-rwx------. 1 azetner grp_azetner    42 Dec  1 18:45 sleepytest.out
-rwx------. 1 azetner grp_azetner  1176 Dec  1 18:09 Screen.ipynb
-rwx------. 1 azetner grp_azetner    38 Nov 29 12:00 README.md

In [13]:
# Exercise 3.3
# Edit all parts before running cell
# Choose an appropriate partition on which to run this job
# Use one cpu and 1G of memory
# Edit the job output name to include your username and the job ID: username-jobid.out
# Edit the job name to be FileNamingTest-USERNAME
# grep for your username

sbatch -p NMLResearch -c 1 --mem=1G -J FileNamingTest-azetner -o %u-%j.out --wrap="sleep 30; ls"
sleep 5
sq | grep azetner
ls -lr

Submitted batch job 2788030
2788030  NMLResear  FileNamingTest-azetner                              azetner   RUNNING  0:05        UNLIMITED     1      waffles-b-10      1     1G
total 1000
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:47 waitstart.out
drwx------. 3 azetner grp_azetner    21 Dec  1 18:35 testing
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:45 test90.out
drwx------. 2 azetner grp_azetner    45 Nov 29 11:58 sources
-rwx---r-x. 1 azetner grp_azetner 11406 Nov  2 16:31 snakemake_basecall_tutorial.ipynb
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:50 slurm-2788007.out
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:50 slurm-2788006.out
-rwx------. 1 azetner grp_azetner     0 Dec  1 18:47 slurm-2788004.out
-rwx------. 1 azetner grp_azetner   106 Dec  1 18:44 slurm-2787974.out
-rwx------. 1 azetner grp_azetner   106 Dec  1 18:43 slurm-2787971.out
-rwx------. 1 azetner grp_azetner     5 Dec  1 18:41 slurm-2787968.out
-rwx------. 1 azetner grp_azetner    42 Dec 

In [15]:
# Exercise 3.4
# When 3.3 is complete
# Use sacct to see how many cpus your job used and what state it is in

sacct -j 2788030

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
2788030      FileNamin+ NMLResear+                     1  COMPLETED      0:0 
2788030.bat+      batch                                1  COMPLETED      0:0 


### 2. Resource Management

#### Partitions

They are groupings of allocated resources:
  * OutbreakResponse
  * NMLResearch
  * ExternalResearch
  * Reference/Surveillance

**They are not priorities or queues**  
Used to allocate resources based on how much people are doing each of the above types of jobs
 * Using the correct one for your work will result in more allocated resources to that queue

So please use the right one :)


#### Determining Required Resources

As to not despoil the beautiful pasture of Waffles2.0 we need to use what resources we need and not (too much) more. Remember to only use one core unless your tool specifically calls for more. See ```sbatch``` above.

How to do that?

**A Primer on Estimating Resources**  
 1. Run something for the first time with 1 core and 1G of memory
 2. Did it work? Yes? Move on. No? Double the memory and try again.
 3. Run ```sacct``` to see how much it actually used for resources
 4. Record that and use the given numbers in the future, padded slightly (~5%)

**A Good sacct Command**  
sacct -j JOBID --format="JobID,JobName,Partition,ReqCPUs,ReqMem,AllocCPUs,MaxRSS"
 * Identifying details about the job (ID, Name, Partition)
 * Resources requested and used
 
**A Note on External Resources**
Some tools (eg NCBI Entrez Direct) rely on connections to external databases to run.  
Those same external databases often limit the number of simultaneous connections from one client.  
This can result in all NML requests being blocked when a user submits too many simultaneous connections. 

To avoid this:  
 * Batch retrieve from external resources, then use local copies for subsequent jobs
 * Scale up slowly: one job, then 20, 50, 100, etc... Not 500 to start!


In [16]:
# Exercise 4.0

# Use sacct to see how many resources your earlier (3.3) job used
# Add the format argument above to show:
# JobID,JobName,Partition,ReqCPUs,ReqMem,AllocCPUs,MaxRSS

sacct -j 2788030 --format="JobID,JobName,Partition,ReqCPUs,ReqMem,AllocCPUs,MaxRSS"

       JobID    JobName  Partition  ReqCPUS     ReqMem  AllocCPUS     MaxRSS 
------------ ---------- ---------- -------- ---------- ---------- ---------- 
2788030      FileNamin+ NMLResear+        1        1Gn          1            
2788030.bat+      batch                   1        1Gn          1      1.50M 


Note that the job you requested 1 gigabyte of memory used nearly 1000x less than that

In [20]:
# Exercise 5.1a: Set Up
# We're going to look at troubleshooting failing jobs...
# Run the following cell. It's going to do the following:

# Activate a conda environment containing SPAdes
source activate spades

# Submit a job: in this case, a test run for spades
# For testing purposes, SPAdes comes with a toy data set (reads that align to first 1000 bp of E. coli)
sbatch -p NMLResearch -c 1 -o EXspades.out --mem 48M --wrap="spades.py --test"

# Wait a bit...
sleep 90

(/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) Submitted batch job 2788076
(/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) 

: 1

Note that Jupyter does some strange things with conda output and will sometimes repeat the environment name in the code cell output. Nothing to worry about

In [21]:
# Exercise 5.1b: 
# Display the tail end of the output
# If it's not done you may have to re-run this cell
tail EXspades.out

(/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades)   0:00:24.354    64M / 8G    INFO   StageManager             (stage.cpp                 : 126)   STAGE == Gap Closer
  0:00:24.354    64M / 8G    INFO    General                 (graph_pack.hpp            :  99)   Index refill
  0:00:24.357    64M / 8G    INFO   K-mer Index Building     (kmer_index_builder.hpp    : 289)   Building kmer index
  0:00:24.357    64M / 8G    INFO    General                 (kmer_index_builder.hpp    : 106)   Splitting kmer instances into 256 buckets using 16 threads. This might take a while.
  0:00:24.357    64M / 8G    INFO    General                 (file_limit.hpp            :  32)   Open file limit set to 1024
  0:00:24.357    64M / 8G    INFO    General                 (kmer_splitters.hpp        :  74)   Memory available for splitting buffers: 1.29036 Gb
  0:00:24.357    64M / 8G    INFO    General                 (kmer_splitters.hpp        :  82)   Using cell siz

: 1

In [22]:
# Exercise 5.2: Your turn
# Look at the output above, specifically the last 3 lines
# Why did the job fail?
# How much memory did it require?
# Rewrite the above sbatch command with more memory

sbatch -p NMLResearch -c 1 -o EXspades.out --mem 96M --wrap="spades.py --test"

sleep 120
# While it sleeps why not hop over to your terminal and use
# tail -f EXspades.out to watch if run
# or use your knowledge of sq | grep to watch your job

tail EXspades.out


(/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) Submitted batch job 2788092
(/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades)  * Paths in the assembly graph corresponding to the contigs are in /Drives/K/AZetner/jupyter_notebooks/spades_test/contigs.paths
 * Paths in the assembly graph corresponding to the scaffolds are in /Drives/K/AZetner/jupyter_notebooks/spades_test/scaffolds.paths

======= SPAdes pipeline finished.

========= TEST PASSED CORRECTLY.

SPAdes log can be found here: /Drives/K/AZetner/jupyter_notebooks/spades_test/spades.log

Thank you for using SPAdes!
(/Drives/P/conda_envs/spades) 

: 1

## IV. Conda
### 1. Re-introduction to Conda

#### What Conda Does
* Package and environment manager
* Simplifies installation and management of software packages
* Silos those installations into non-overlapping environments

#### Why Use Conda
* Tool dependencies are handled in their respective environments, preventing incompatibilities and simplifying installation
* Users can test software without waiting for support

#### How to use Conda? 
* The whole next section
* Will be focused on using Conda on Waffles
* Use proper cluster etiquette throughout



### 2. Managing Environments
#### List Existing Environments
```conda env list```

* Can be run on the head node (no sbatch/srun needed)
* Active environment identified with ```*```
* \[Environment Name\] \[Filesystem Location\]
* Base and all environments on the P drive are not editable
* All environments you create are available for editing


In [23]:
# Example

# List the environments

conda env list

(/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) # conda environments:
#
albacore                 /Drives/P/conda_envs/albacore
albacore-2.2.7           /Drives/P/conda_envs/albacore-2.2.7
albacore-2.3.0           /Drives/P/conda_envs/albacore-2.3.0
albacore-2.3.3           /Drives/P/conda_envs/albacore-2.3.3
albacore-dev             /Drives/P/conda_envs/albacore-dev
beast-2.4.5              /Drives/P/conda_envs/beast-2.4.5
beast-2.5.0              /Drives/P/conda_envs/beast-2.5.0
bioperl                  /Drives/P/conda_envs/bioperl
blast_plus_2.7.1         /Drives/P/conda_envs/blast_plus_2.7.1
bowtie2                  /Drives/P/conda_envs/bowtie2
bracken-1.0              /Drives/P/conda_envs/bracken-1.0
cdc_nomenclature         /Drives/P/conda_envs/cdc_nomenclature
circlator                /Drives/P/conda_envs/circlator
ectyper-0.5.4            /Drives/P/conda_envs/ectyper-0.5.4
graphlan                 /Drives/P/

: 1

#### Activate Existing Environments
```source activate ENVNAME```

* Can be run on the head node (no sbatch/srun needed)
* All software in the environment is now available for the user
* Packages install to the active environment
* Waffles terminal prompt becomes:  
```(ENVIRONMENT) [USER@waffles DIRECTORY]```\$

In [24]:
# Exercise 6
# List the environments as above
conda env list

(/Drives/P/conda_envs/spades) (/Drives/P/conda_envs/spades) # conda environments:
#
albacore                 /Drives/P/conda_envs/albacore
albacore-2.2.7           /Drives/P/conda_envs/albacore-2.2.7
albacore-2.3.0           /Drives/P/conda_envs/albacore-2.3.0
albacore-2.3.3           /Drives/P/conda_envs/albacore-2.3.3
albacore-dev             /Drives/P/conda_envs/albacore-dev
beast-2.4.5              /Drives/P/conda_envs/beast-2.4.5
beast-2.5.0              /Drives/P/conda_envs/beast-2.5.0
bioperl                  /Drives/P/conda_envs/bioperl
blast_plus_2.7.1         /Drives/P/conda_envs/blast_plus_2.7.1
bowtie2                  /Drives/P/conda_envs/bowtie2
bracken-1.0              /Drives/P/conda_envs/bracken-1.0
cdc_nomenclature         /Drives/P/conda_envs/cdc_nomenclature
circlator                /Drives/P/conda_envs/circlator
ectyper-0.5.4            /Drives/P/conda_envs/ectyper-0.5.4
graphlan                 /Drives/P/conda_envs/graphlan
groot                    /Drives/P/conda

: 1

In [1]:
# Exercise 7.1
# What environment are you currently in?



     active environment : None
       user config file : /home/CSCScience.ca/AZetner/.condarc
 populated config files : /opt/miniconda3/.condarc
          conda version : 4.5.11
    conda-build version : not installed
         python version : 3.6.2.final.0
       base environment : /opt/miniconda3  (read only)
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
                          https://conda.anaconda.org/bioconda/linux-64
                          https://conda.anaconda.org/bioconda/noarch
                          https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/free/linux-64
                          https://repo.anaconda.com/pkgs/free/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/

In [2]:
# Activate the environment samtools with source activate
source activate samtools

(samtools) 

: 1

#### Show Information About Current Environment
```conda info```

* Can be run on the head node (no sbatch/srun needed)
* Shell level: are you in a nested environment? 0 if base, 1 if not, 2 if so.
* Config file locations
* Environment locations
* Other information

```conda list```

* Can be run on the head node (no sbatch/srun needed)
* Shows what packages are installed and from what channel
* Add the ```--revisions``` flag to show installed updates
  * Updates to the environment are saved as "revisions" 
  * Includes any new installs etc...


In [3]:
# Exercise 8.1
# Show the environment information using conda info
# Find out what shell level are we in?
# Find out where the current environments are located
conda info

(samtools) (samtools) (samtools) (samtools) 
     active environment : samtools
    active env location : /home/CSCScience.ca/AZetner/.conda/envs/samtools
            shell level : 1
       user config file : /home/CSCScience.ca/AZetner/.condarc
 populated config files : /opt/miniconda3/.condarc
          conda version : 4.5.11
    conda-build version : not installed
         python version : 3.6.2.final.0
       base environment : /opt/miniconda3  (read only)
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
                          https://conda.anaconda.org/bioconda/linux-64
                          https://conda.anaconda.org/bioconda/noarch
                          https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/free/linux-64
                          https://repo

: 1

In [29]:
# Exercise 8.2
# Activate the base environment
# What revision is it on?
source activate base
conda list --revisions

(base) (base) (base) (base) 2018-07-24 10:03:38  (rev 0)
    +asn1crypto-0.22.0
    +ca-certificates-2017.08.26
    +certifi-2017.7.27.1
    +cffi-1.10.0
    +chardet-3.0.4
    +conda-4.3.27
    +conda-env-2.6.0
    +cryptography-2.0.3
    +idna-2.6
    +libedit-3.1
    +libffi-3.2.1
    +libgcc-ng-7.2.0
    +libstdcxx-ng-7.2.0
    +ncurses-6.0
    +openssl-1.0.2l
    +pip-9.0.1
    +pycosat-0.6.2
    +pycparser-2.18
    +pyopenssl-17.2.0
    +pysocks-1.6.7
    +python-3.6.2
    +readline-7.0
    +requests-2.18.4
    +ruamel_yaml-0.11.14
    +setuptools-36.5.0
    +six-1.10.0
    +sqlite-3.20.1
    +tk-8.6.7
    +urllib3-1.22
    +wheel-0.29.0
    +xz-5.2.3
    +yaml-0.1.7
    +zlib-1.2.11

2018-07-24 10:04:21  (rev 1)
     conda  {4.3.27 -> 4.5.8 (conda-forge)}
     conda-env  {2.6.0 -> 2.6.0 (conda-forge)}
     pycosat  {0.6.2 -> 0.6.3 (conda-forge)}
    +appdirs-1.4.3 (conda-forge)
    +attrs-18.1.0 (conda-forge)
    +automat-0.7.0 (conda-forge)
    +backcall-0.1.0 (conda-forge)
   

: 1

#### Deactivate Current Environment
```source deactivate```

* Returns to previous environment
* The base environment is also called shell 0 and None
* Environments can be nested within each other
 * Nesting results in packages shared from higher env to lower
 * Packages found in both are superceded by the newest 


In [22]:
# Run this cell until you are in your base environment
# Base is also referred to as shell level 0 or None
 source deactivate
 conda info
 


     active environment : None
            shell level : 0
       user config file : /home/CSCScience.ca/AZetner/.condarc
 populated config files : /opt/miniconda3/.condarc
          conda version : 4.5.11
    conda-build version : not installed
         python version : 3.6.2.final.0
       base environment : /opt/miniconda3  (read only)
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
                          https://conda.anaconda.org/bioconda/linux-64
                          https://conda.anaconda.org/bioconda/noarch
                          https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/free/linux-64
                          https://repo.anaconda.com/pkgs/free/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                        

In [18]:
# Example 
# This uses an environment y'all don't have so sit back and watch
# What version is python in base?
echo "Base environment:"
python -V
# Activate the python 2.7 env
echo "2.7 environment:"
source activate python2.7
python -V
echo "SPAdes environment:"
source activate spades
python -V


Base environment:
Python 3.6.2 :: Anaconda, Inc.
2.7 environment:
(python2.7) Python 2.7.15
(python2.7) SPAdes environment:
(python2.7) (/Drives/P/conda_envs/spades) Python 3.6.5 :: Anaconda, Inc.
(/Drives/P/conda_envs/spades) 

: 1

In [39]:
# Exercise 9
# Return to the previous environment by deactivating the current one
# What environment are you in now? What shell level? Use conda info
# Deactivate further (by running this cell) until you reach the base environment

source deactivate
conda info





     active environment : None
            shell level : 0
       user config file : /home/CSCScience.ca/AZetner/.condarc
 populated config files : /opt/miniconda3/.condarc
          conda version : 4.5.11
    conda-build version : not installed
         python version : 3.6.2.final.0
       base environment : /opt/miniconda3  (read only)
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
                          https://conda.anaconda.org/bioconda/linux-64
                          https://conda.anaconda.org/bioconda/noarch
                          https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/free/linux-64
                          https://repo.anaconda.com/pkgs/free/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                        

#### Creating Environments
```conda create OTHER-ARGUMENTS -n ENVNAME```

* Not allowed to run on the head node (use sbatch/srun)
* Important arguments:
 * ```-y```: flag, accept all prompts
 * ```-n / --name```: name your environment, no spaces
 * ```--clone ENVNAME```: clone an existing environment
   * _Currently broken for unknown reason_ use ```--explicit``` method below
 * ```-f ENVCONFIG.yml```: clone an existing environment from a configuration file
 * After the environment creation arguments, a list of package names can be given and installed at the same time


In [34]:
# Exercise 10.1
# Create a new environment by modifying the sbatch command below
# Name the environment testenv-YOURNAME

sbatch -p NMLResearch -c 1 --mem 1G --wrap="conda create -y -n testenv-azetner"

(testenv-azetner) (testenv-azetner) (testenv-azetner) (testenv-azetner) Submitted batch job 2812198
(testenv-azetner) 

: 1

In [35]:
# Exercise 10.2
# View your newly created environment with the others using conda env list
conda env list


(testenv-azetner) (testenv-azetner) # conda environments:
#
albacore                 /Drives/P/conda_envs/albacore
albacore-2.2.7           /Drives/P/conda_envs/albacore-2.2.7
albacore-2.3.0           /Drives/P/conda_envs/albacore-2.3.0
albacore-2.3.3           /Drives/P/conda_envs/albacore-2.3.3
albacore-dev             /Drives/P/conda_envs/albacore-dev
beast-2.4.5              /Drives/P/conda_envs/beast-2.4.5
beast-2.5.0              /Drives/P/conda_envs/beast-2.5.0
bioperl                  /Drives/P/conda_envs/bioperl
blast_plus_2.7.1         /Drives/P/conda_envs/blast_plus_2.7.1
bowtie2                  /Drives/P/conda_envs/bowtie2
bracken-1.0              /Drives/P/conda_envs/bracken-1.0
cdc_nomenclature         /Drives/P/conda_envs/cdc_nomenclature
circlator                /Drives/P/conda_envs/circlator
ectyper-0.5.4            /Drives/P/conda_envs/ectyper-0.5.4
graphlan                 /Drives/P/conda_envs/graphlan
groot                    /Drives/P/conda_envs/groot
hclust2     

: 1

In [36]:
# Exercise 10.3
# Examine the slurm output from above with the more command

more slurm-2812198.out

(testenv-azetner) (testenv-azetner) (testenv-azetner) Solving environment: ...working... done

## Package Plan ##

  environment location: /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
#
# To activate this environment, use
#
#     $ conda activate testenv-azetner
#
# To deactivate an active environment, use
#
#     $ conda deactivate

(testenv-azetner) 

: 1

In [37]:
# Exercise 10.4
# Activate your new environment
# Use the correct conda command to show INFOrmation about it

source activate testenv-azetner
conda info

(testenv-azetner) (testenv-azetner) (testenv-azetner) (testenv-azetner) (testenv-azetner) 
     active environment : testenv-azetner
    active env location : /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner
            shell level : 1
       user config file : /home/CSCScience.ca/AZetner/.condarc
 populated config files : /opt/miniconda3/.condarc
          conda version : 4.5.11
    conda-build version : not installed
         python version : 3.6.2.final.0
       base environment : /opt/miniconda3  (read only)
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
                          https://conda.anaconda.org/bioconda/linux-64
                          https://conda.anaconda.org/bioconda/noarch
                          https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.c

: 1

In [39]:
# Exercise 10.5
# Show installed packages using the appropriate command
conda list

# Modify that command to show revisions
conda list --revisions

(testenv-azetner) (testenv-azetner) # packages in environment at /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner:
#
# Name                    Version                   Build  Channel
(testenv-azetner) (testenv-azetner) (testenv-azetner) 2018-12-03 22:17:35  (rev 0)

(testenv-azetner) 

: 1

Nothing shows up, nothing added beyond creating the environment

#### Save Explicit Environment Information to a Text File
```conda list --explicit > bio-env.txt```

* Saves instructions to recreate that environment later
* Does not save revision history


#### Create Environment from Text File
```conda create -y -n bio-env --file bio-env.txt```

* Uses the instructions as created to recreate an environment
* No revision history is included in this method
* We're trying to fix --clone not working. Until then, use this.



In [44]:
# Exercise 11.1
# First copy /Drives/W/Temporary/conda-workshop-2018/python2.txt

# cp /Drives/W/Temporary/conda-workshop-2018/python2.txt ./


In [45]:
# Exercise 11.2
# Make a new environment from that file
# Use the below command in wrap
# conda create -y -n mypy2 --file python2.txt 
sbatch -p NMLResearch -c 1 --mem 1G --wrap="conda create -y -n mypy2 --file python2.txt"

Submitted batch job 2813370


In [46]:
cat slurm-2813370.out

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [47]:
# Exercise 11.3
# Activate that environment
# List revisions 
source activate mypy2
conda list --revisions



(mypy2) 2018-12-03 22:31:40  (rev 0)
    +bzip2-1.0.6 (conda-forge)
    +ca-certificates-2018.10.15 (conda-forge)
    +certifi-2018.10.15 (conda-forge)
    +libffi-3.2.1 (conda-forge)
    +libgcc-ng-7.2.0 (conda-forge)
    +libstdcxx-ng-7.2.0 (conda-forge)
    +ncurses-6.1 (conda-forge)
    +openssl-1.0.2p (conda-forge)
    +pip-18.1 (conda-forge)
    +python-3.7.1 (conda-forge)
    +readline-7.0 (conda-forge)
    +setuptools-40.6.2 (conda-forge)
    +sqlite-3.25.3 (conda-forge)
    +tk-8.6.9 (conda-forge)
    +wheel-0.32.3 (conda-forge)
    +xz-5.2.4 (conda-forge)
    +zlib-1.2.11 (conda-forge)

(mypy2) 

: 1

These 'revisions' on rev 0 were the instructions on how conda should create the environment.

####  Removing Environments
```conda env remove -y --name ENVIRONMENTNAME```

* Use sbatch
* Use ```-y``` in jupyter
* Deletes the environment
* No take-backsies
* Recommend saving the info explicitly if any chance of needing again


In [48]:
# Exercise 12.1
# Activate mypy2 if not already
# Save explicit info to file mypy2.txt
conda list --explicit > mypy2.txt



(mypy2) (mypy2) (mypy2) (mypy2) 

: 1

In [49]:
# Exercise 12.2
# Remove that environment 

conda env remove -y --name mypy2
#sbatch -p NMLResearch -c 1 --mem=1G --wrap="conda env remove -y --name mypy2"

# Did it work? 

(mypy2) (mypy2) (mypy2) 
CondaEnvironmentError: cannot remove current environment. deactivate and run conda remove again

(mypy2) (mypy2) (mypy2) 

: 1

In [51]:
# Exercise 12.3
# If it didn't work, remedy the situation below based on output from the cell above
# Try again

source deactivate
sbatch -p NMLResearch -c 1 --mem=1G --wrap="conda env remove -y --name mypy2"



Submitted batch job 2813648


### 3. Managing packages

#### Get Packages from Channels
* Directories of packages
* URLs for those directories
* Many defaults: r/pro/free/main
* Custom channels available

In [1]:
# Example 
conda config --show channels

channels:
  - conda-forge
  - bioconda
  - defaults


#### Show Packages in the Environment
```conda list```

* Table of packages
* Name, Version, Build, Channel
* Defaults to active environment
* use ```-n``` argument to list from a different one

In [2]:
# Example
conda list -n mash

# packages in environment at /Drives/P/conda_envs/mash:
#
# Name                    Version                   Build  Channel
capnproto                 0.6.1                         0    conda-forge
gsl                       2.2.1                h0c605f7_3    anaconda
libgcc                    7.2.0                h69d50b8_2    anaconda
libgcc-ng                 7.2.0                hdf63c60_3    anaconda
libgfortran-ng            7.2.0                hdf63c60_3    anaconda
libopenblas               0.2.20               h9ac9557_7    anaconda
libstdcxx-ng              7.2.0                hdf63c60_3    anaconda
mash                      2.0                    gsl2.2_2    bioconda
openblas                  0.2.20                        4    anaconda
openblas-devel            0.2.20                        7    anaconda
zlib                      1.2.11               ha838bed_2    anaconda


#### Installing Packages to the Environment
```conda install PACKAGENAME```

* Must be done with sbatch!
* Default install to active environment 
  * Use ```-n``` argument to install to a different one
* Installs the newest version by default
* Updates or installs dependencies if possible
* Use ```-y``` to accept prompts as it runs
* A custom channel can be supplied with ```-c```


In [52]:
# Example
# Install mash to testenv-azetner
# The channel command is optional because bioconda is one of our custom channels

sbatch -p NMLResearch -c 1 --mem=1G -o mash-install-progress.out --wrap="conda install -y -c bioconda -n testenv-azetner mash"

# Watch that in the terminal with: tail -f mash-install-progress.out


Submitted batch job 2813698


In [ ]:
# Exercise 13
# Install mash to your own testenv-YOURNAME
# Watch that in the terminal with: tail -f mash-install-progress.out


#### Installing Specific Versions
 * Newest: numpy - Results in newest available in channels
 * Fuzzy: numpy=1.11 - Results in 1.11.0, 1.11.1, 1.11.2, 1.11.18 etc...
 * Exact: numpy=1.11.0 - Results in 1.11.0
 * Greater than or equal to: "numpy>=1.11" - Results in 1.11.0 or higher
 
#### Updating Existing Packages 
```conda update PACKAGENAME```

* Must be done with sbatch!
* Will kick out if dependency updates required as well
    * Include ```--update-deps``` if needed
    * Attempts to update the package along with its dependencies
* Use ```-y``` to accept prompts as it runs

#### Finding Available Conda Packages
```conda search PACKAGENAME```

* Google "conda SOFTWARENAME"
 * eg. https://anaconda.org/bioconda/shovill
* Search on the command line:  
 * For the package: ```conda search shovill```   
 * For a version: ```conda search "shovill>=1.0"```  
   * Uses the rules above
 * Display info: ```conda info shovill=1.0.4```

 

In [23]:
# Exercise 14.1
# Search for bbmap 

conda search bbmap

Loading channels: done
# Name                  Version           Build  Channel             
bbmap                     35.85               1  bioconda            
bbmap                     35.85               2  bioconda            
bbmap                     36.32               0  bioconda            
bbmap                     36.84               0  bioconda            
bbmap                     37.02               0  bioconda            
bbmap                     37.10               0  bioconda            
bbmap                     37.10               1  bioconda            
bbmap                     37.17               0  bioconda            
bbmap                     37.17               1  bioconda            
bbmap                     37.52               0  bioconda            
bbmap                     37.52               1  bioconda            
bbmap                     37.62               0  bioconda            
bbmap                     37.62               1  bioconda          

In [24]:
# Exercise 14.2
# Display info for bbmap version 38.22

conda info bbmap=38.22


bbmap 38.22 h470a237_0
----------------------
file name   : bbmap-38.22-h470a237_0.tar.bz2
name        : bbmap
version     : 38.22
build string: h470a237_0
build number: 0
channel     : https://conda.anaconda.org/bioconda/linux-64
size        : 7.3 MB
arch        : x86_64
constrains  : ()
license     : UC-LBL license (see package)
md5         : 44ae15e33374723ec86df7d7a1748258
platform    : linux
subdir      : linux-64
timestamp   : 1534837421824
url         : https://conda.anaconda.org/bioconda/linux-64/bbmap-38.22-h470a237_0.tar.bz2
dependencies:
    libgcc-ng >=4.9
    openjdk >=7.0


In [25]:
# Exercise 14.3
# Install bbmap version 38.22 to your testenv
# Either by activating it or using the -n argument

sbatch -p NMLResearch -c 1 --mem 1G --wrap="conda install -y -n testenv-azetner bbmap=38.22"

Submitted batch job 2809308


In [26]:
more slurm-2809308.out

Solving environment: ...working... done

## Package Plan ##

  environment location: /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner

  added / updated specs: 
    - bbmap=38.22


The following NEW packages will be INSTALLED:

    bbmap: 38.22-h470a237_0 bioconda

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


#### Removing packages

```conda remove -y PACKAGENAME```

* Run via sbatch!
* Removes the associated dependencies as well


In [27]:
# Exercise 15
# Activate your testenv or call it from the remove command
# Remove bbmap
# source activate testenv-azetner
# sbatch -p NMLResearch -c 1 --mem 1G --wrap="conda remove -y bbmap"

sbatch -p NMLResearch -c 1 --mem 1G -o bbmap-removal-%j.out --wrap="conda remove -y -n testenv-azetner bbmap"

Submitted batch job 2809336


In [28]:
more bbmap-removal-*

::::::::::::::
bbmap-removal-2804669.out
::::::::::::::
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner

  removed specs: 
    - bbmap


The following packages will be REMOVED:

    bbmap: 38.22-h470a237_0 bioconda

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [55]:
# Exercise 15.1
# Activate testenv-YOURNAME
source activate testenv-azetner
# List packages
conda list 

(testenv-azetner) (testenv-azetner) # packages in environment at /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner:
#
# Name                    Version                   Build  Channel
(testenv-azetner) 

: 1

In [18]:
# Exercise 15.2
# Search for versions of shovill
conda search shovill

(testenv-azetner) (testenv-azetner) Loading channels: done
# Name                  Version           Build  Channel             
shovill                   0.7.1      pl5.22.0_0  bioconda            
shovill                   0.7.1      pl5.22.0_1  bioconda            
shovill                   0.7.1      pl5.22.0_2  bioconda            
shovill                   0.8.0      pl5.22.0_0  bioconda            
shovill                   0.9.0               1  bioconda            
shovill                   0.9.0      pl5.22.0_0  bioconda            
shovill                   1.0.0               0  bioconda            
shovill                   1.0.1               0  bioconda            
shovill                   1.0.4               0  bioconda            
(testenv-azetner) 

: 1

In [56]:
# Exercise 15.3
# Install version 0.9.0 of shovill
sbatch -p NMLResearch -c 1 --mem 1G --wrap="conda install shovill=0.9.0"

(testenv-azetner) (testenv-azetner) Submitted batch job 2815664
(testenv-azetner) 

: 1

In [57]:
more slurm-2815664.out

Solving environment: ...working... done

## Package Plan ##

  environment location: /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner

  added / updated specs: 
    - shovill=0.9.0


The following NEW packages will be INSTALLED:

    bwa:             0.7.17-ha92aebf_3       bioconda   
    bzip2:           1.0.6-h470a237_2        conda-forge
    ca-certificates: 2018.11.29-ha4d7672_0   conda-forge
    certifi:         2018.11.29-py37_1000    conda-forge
    curl:            7.62.0-h74213dd_0       conda-forge
    flash:           1.2.11-ha92aebf_2       bioconda   
    kmc:             3.1.1rc1-h76f5088_0     bioconda   
    krb5:            1.16.2-hbb41f41_0       conda-forge
    libcurl:         7.62.0-hbdb9355_0       conda-forge
    libdeflate:      1.0-h470a237_0          bioconda   
    libedit:         3.1.20170329-haf1bffa_1 conda-forge
(testenv-azetner) 

In [58]:
conda list --revisions


2018-12-03 23:11:41  (rev 0)

2018-12-03 23:17:56  (rev 1)
    +bwa-0.7.17 (bioconda)
    +bzip2-1.0.6 (conda-forge)
    +ca-certificates-2018.11.29 (conda-forge)
    +certifi-2018.11.29 (conda-forge)
    +curl-7.62.0 (conda-forge)
    +flash-1.2.11 (bioconda)
    +kmc-3.1.1rc1 (bioconda)
    +krb5-1.16.2 (conda-forge)
    +libcurl-7.62.0 (conda-forge)
    +libdeflate-1.0 (bioconda)
    +libedit-3.1.20170329 (conda-forge)
    +libffi-3.2.1 (conda-forge)
    +libgcc-ng-7.2.0 (conda-forge)
    +libssh2-1.8.0 (conda-forge)
    +libstdcxx-ng-7.2.0 (conda-forge)
    +lighter-1.1.2 (bioconda)
    +ncurses-6.1 (conda-forge)
    +openjdk-11.0.1 (conda-forge)
    +openssl-1.0.2p (conda-forge)
    +perl-5.26.2 (conda-forge)
    +perl-file-spec-3.48_01 (bioconda)
    +perl-findbin-1.51 (bioconda)
    +perl-time-piece-1.27 (bioconda)
    +pigz-2.3.4 (conda-forge)
    +pilon-1.23 (bioconda)
    +pip-18.1 (conda-forge)
    +python-3.7.1 (conda-forge)
    +readline-7.0 (conda-forge)
    +samtools-1.9

: 1

In [61]:
# Exercise 15.4
# Update shovill to latest version
# If you get this message you forgot something important: # All requested packages already installed.
sbatch -p NMLResearch -c 1 --mem 1G -o shovill-update-%j.out --wrap="conda update -y --update-deps shovill"

(testenv-azetner) (testenv-azetner) Submitted batch job 2815804
(testenv-azetner) 

: 1

In [62]:
more shovill-update*

Solving environment: ...working... done
boost-1.67.0         | 316 KB    | ########## | 100% 
boost-cpp-1.67.0     | 19.7 MB   | 6          | 
boost-cpp-1.67.0     | 19.7 MB   | ########5
boost-cpp-1.67.0     | 19.7 MB   | #####
boost-cpp-1.67.0     | 19.7 MB   | #
boost-cpp-1.67.0     | 19.7 MB  
boost-cpp-1.67.0     | 19.7 
boost-cpp-1.67.0     | 1
boost-cpp-1.67.0    
boost-cpp-1.67.0
boost-cpp-1.
boost-cp
boos
boost-cpp-1.67.0     | 19.7 MB   | #########7 |  97% 
boost-cpp-1.67.0     | 19.7 MB   | ########## | 10
% 
## Package Plan ##

  environment location: /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner

  added / updated specs: 
    - shovill
(testenv-azetner) 

In [63]:
# Exercise 15.5
# Look at environment revisions
conda list --revisions

(testenv-azetner) (testenv-azetner) 2018-12-03 23:11:41  (rev 0)

2018-12-03 23:17:56  (rev 1)
    +bwa-0.7.17 (bioconda)
    +bzip2-1.0.6 (conda-forge)
    +ca-certificates-2018.11.29 (conda-forge)
    +certifi-2018.11.29 (conda-forge)
    +curl-7.62.0 (conda-forge)
    +flash-1.2.11 (bioconda)
    +kmc-3.1.1rc1 (bioconda)
    +krb5-1.16.2 (conda-forge)
    +libcurl-7.62.0 (conda-forge)
    +libdeflate-1.0 (bioconda)
    +libedit-3.1.20170329 (conda-forge)
    +libffi-3.2.1 (conda-forge)
    +libgcc-ng-7.2.0 (conda-forge)
    +libssh2-1.8.0 (conda-forge)
    +libstdcxx-ng-7.2.0 (conda-forge)
    +lighter-1.1.2 (bioconda)
    +ncurses-6.1 (conda-forge)
    +openjdk-11.0.1 (conda-forge)
    +openssl-1.0.2p (conda-forge)
    +perl-5.26.2 (conda-forge)
    +perl-file-spec-3.48_01 (bioconda)
    +perl-findbin-1.51 (bioconda)
    +perl-time-piece-1.27 (bioconda)
    +pigz-2.3.4 (conda-forge)
    +pilon-1.23 (bioconda)
    +pip-18.1 (conda-forge)
    +python-3.7.1 (conda-forge)
    +readline

: 1

In [64]:
# Exercise 15.6
# Install back to first revision
# What version of shovill is installed?

sbatch -p NMLResearch -c 1 --mem 1G -o back-a-revision-%j.out --wrap="conda install -y --revision 1; conda list"

(testenv-azetner) (testenv-azetner) (testenv-azetner) (testenv-azetner) Submitted batch job 2815859
(testenv-azetner) 

: 1

In [65]:
more back-a-rev*


## Package Plan ##

  environment location: /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner

  added / updated specs: 
    - shovill


The following packages will be REMOVED:

    blas:        1.1-openblas                        conda-forge
    boost:       1.67.0-py36h3e44d54_0               conda-forge
    boost-cpp:   1.67.0-h3a22d5f_0                   conda-forge
    capnproto:   0.6.1-hfc679d8_1                    conda-forge
    gsl:         2.2.1-blas_openblas_4               conda-forge
    icu:         58.2-hfc679d8_0                     conda-forge
    libgcc:      7.2.0-h69d50b8_2                    conda-forge
    libgfortran: 3.0.0-1                             conda-forge
    mash:        2.1-h9dd4a16_0                      bioconda   
    megahit:     1.1.3-py36_0                        bioconda   
    numpy:       1.15.1-py36_blas_openblashd3ea46f_0 conda-forge
    openblas:    0.2.20-8                            conda-forge
(testenv-azetner) 

In [66]:
conda list

# packages in environment at /home/CSCScience.ca/AZetner/.conda/envs/testenv-azetner:
#
# Name                    Version                   Build  Channel
bwa                       0.7.17               ha92aebf_3    bioconda
bzip2                     1.0.6                h470a237_2    conda-forge
ca-certificates           2018.11.29           ha4d7672_0    conda-forge
certifi                   2018.11.29            py37_1000    conda-forge
curl                      7.62.0               h74213dd_0    conda-forge
flash                     1.2.11               ha92aebf_2    bioconda
kmc                       3.1.1rc1             h76f5088_0    bioconda
krb5                      1.16.2               hbb41f41_0    conda-forge
libcurl                   7.62.0               hbdb9355_0    conda-forge
libdeflate                1.0                  h470a237_0    bioconda
libedit                   3.1.20170329         haf1bffa_1    conda-forge
libffi                    3.2.1                hfc679d

: 1

In [67]:
conda list --revisions

2018-12-03 23:11:41  (rev 0)

2018-12-03 23:17:56  (rev 1)
    +bwa-0.7.17 (bioconda)
    +bzip2-1.0.6 (conda-forge)
    +ca-certificates-2018.11.29 (conda-forge)
    +certifi-2018.11.29 (conda-forge)
    +curl-7.62.0 (conda-forge)
    +flash-1.2.11 (bioconda)
    +kmc-3.1.1rc1 (bioconda)
    +krb5-1.16.2 (conda-forge)
    +libcurl-7.62.0 (conda-forge)
    +libdeflate-1.0 (bioconda)
    +libedit-3.1.20170329 (conda-forge)
    +libffi-3.2.1 (conda-forge)
    +libgcc-ng-7.2.0 (conda-forge)
    +libssh2-1.8.0 (conda-forge)
    +libstdcxx-ng-7.2.0 (conda-forge)
    +lighter-1.1.2 (bioconda)
    +ncurses-6.1 (conda-forge)
    +openjdk-11.0.1 (conda-forge)
    +openssl-1.0.2p (conda-forge)
    +perl-5.26.2 (conda-forge)
    +perl-file-spec-3.48_01 (bioconda)
    +perl-findbin-1.51 (bioconda)
    +perl-time-piece-1.27 (bioconda)
    +pigz-2.3.4 (conda-forge)
    +pilon-1.23 (bioconda)
    +pip-18.1 (conda-forge)
    +python-3.7.1 (conda-forge)
    +readline-7.0 (conda-forge)
    +samtools-1.9

: 1

## V. Appendix

### Useful *sbatch* command
sbatch -p NMLResearch -c 1 --mem=1G --wrap=""

### Useful *sacct* command
sacct -j  --format="JobID,JobName,Partition,ReqCPUs,ReqMem,AllocCPUs,MaxRSS"


## VI. Screen
Will give a presentation on screen at next BWG. Until then if you're interested check this out:
https://nathan.chantrell.net/linux/an-introduction-to-screen/
